<a href="https://colab.research.google.com/github/raphi-web/spatial_object_detection/blob/main/Object_Detection_ala_raphi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Raphis super awsome Object-Detection colab, 
# not at all a mix of  copy past from other sources! 

# Chane your runtime to GPU for performance!

## How it works,
Basically I use pre trained models from the [tensor flow the model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md) too train a classifier. Oh yeah and I had to completely rewrite a this [Colab](https://colab.research.google.com/github/Tony607/object_detection_demo/blob/master/tensorflow_object_detection_training_colab.ipynb). Because it 1. too complicated and 2. made for tensorflow 1 not the version 2 that is now in colab.

### For spatial detection:
Spatial Datasets are very large pictures easily feature more than 5000 pixels. For this reason, first you have to tile your image. **You can use qgis to export the tiles as vtr layer**. 

You can convert the tif files with the code in my github repository:
https://github.com/raphi-web/spatial_object_detection 

Then it gets trained by 



In [ ]:
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib pycocotools tf_slim lvis tensorflow_addons

In [2]:
!git clone https://github.com/tensorflow/models.git -q
! cd models/research && protoc object_detection/protos/*.proto --python_out=.

fatal: destination path 'models' already exists and is not an empty directory.


In [3]:
import os
import sys
from google.colab import files
#from object_detection.utils import label_map_util
import re
import shutil
import glob
import urllib.request
import tarfile
import tensorflow as tf

In [4]:
num_steps = 20000 
num_eval_steps = 50

1. https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md
2.Faster R-CNN ResNet101 V1 800x1333

In [5]:
MODELS_CONFIG = {
    
        'centernet': {
        'model_name': 'centernet_hg104_512x512_coco17_tpu-8',
        'batch_size':2
    },
        'faster_rcnn': {
        'model_name': 'faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8',
        'batch_size':2
    }

}

In [6]:
num_classes = 1
selected_model = 'centernet'
MODEL = MODELS_CONFIG[selected_model]['model_name']

In [7]:
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/'
DEST_DIR = '/content/loaded_model'



if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

In [8]:
pipeline_file = "loaded_model/pipeline.config"
batch_size = MODELS_CONFIG[selected_model]['batch_size']

In [9]:
# tfrecord upload
train_record_fname = files.upload()
train_record_fname = list(train_record_fname.keys())[0]


Saving craters.tfrecord to craters (1).tfrecord


In [10]:
# label_map_upload
label_map_pbtxt_fname =files.upload()
label_map_pbtxt_fname = list(label_map_pbtxt_fname.keys())[0]

Saving craters_label_map.pbtxt to craters_label_map (1).pbtxt


In [11]:
fine_tune_checkpoint = "loaded_model/checkpoint/ckpt-0"
old_config ="loaded_model/pipeline.config"
new_config = "loaded_model/custom.config"

try:
  os.remove(new_config)
except:
  pass

shutil.copyfile(old_config,new_config)
pipeline_file = new_config

with open(pipeline_file) as f:
    s = f.read()

with open(pipeline_file, 'w') as f:
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)


    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(.tfrecord)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
      
    s = re.sub('fine_tune_checkpoint_type: ".*?"',
               'fine_tune_checkpoint_type: "{}"'.format("detection"), s)


    """
    fine_tune_checkpoint_type: "classification"
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)
    """

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    f.write(s)

pipeline_file = new_config

In [12]:
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
os.environ['PYTHONPATH'] += ":/content/models"
sys.path.append("/content/models")

In [13]:
#!python models/research/object_detection/builders/model_builder_tf2_test.py

In [14]:
model_dir = "results"

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

2021-05-14 19:36:21.269217: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-05-14 19:36:24.894692: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-05-14 19:36:24.908084: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-05-14 19:36:24.968871: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-05-14 19:36:24.969469: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-05-14 19:36:24.969510: I tensorflow/stream_executor/platform/default/dso_loade

In [ ]:
fine_tune_checkpoint_type: "classification"

In [ ]:
! rm -r results